# Предсказание уровня заработной платы  

https://archive.ics.uci.edu/ml/datasets/adult

В данном ДЗ вам нужно будет предсказывать уровень заработной платы респондента: больше 50к долларов в год или меньше.

**Описание признаков**:

1. Категорийные признаки

`workclass`: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. Individual work category

`education`: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. Individual's highest education degree

`marital-status`: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. Individual marital status

`occupation`: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. Individual's occupation

`relationship`: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. Individual's relation in a family

`race`: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. Race of Individual

`sex`: Female, Male.

`native-country`: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands. Individual's native country

2. Количественные признаки

`age`: continuous. Age of an individual

`fnlwgt`: final weight, continuous. The weights on the CPS files are controlled to independent estimates of the civilian noninstitutional population of the US. These are prepared monthly for us by Population Division here at the Census Bureau.

`capital-gain`: continuous.

`capital-loss`: continuous.

`hours-per-week`: continuous. Individual's working hour per week

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
os.chdir("/Users/iakubovskii/Machine_Learning/RANEPA/Fintech_2020/Машинное обучение/Данные/ДЗ1/")
adult = pd.read_csv("adult.csv", dtype = {"fnlwgt": int,
                                             "Education_Num": int,
                                             "Capital_Gain":int,
                                             "Capital_Loss":int,
                                             "Hours_per_week":int,
                                             "Age":int})

In [2]:
# Список категориальных признаков
cat_cols = ['Workclass', 'Education', 'Marital_Status', 'Occupation',
            'Relationship', 'Race', 'Sex', 'Country']

# Список количественных признаков
quant_cols = list(set(adult.columns) - set(cat_cols))
quant_cols.remove("Target")
# Удалим лишние пробелы перед значениями внутри категориальных столбцов
for cat_col in cat_cols:
    adult[cat_col] = adult[cat_col].str.strip(" ")


# Преобразуем категориальные признаки при помощи дамми-кодирования
new_df = pd.concat([adult.drop(cat_cols, axis=1),
                  pd.get_dummies(adult[cat_cols], drop_first=True)],
                 axis=1)
# Удалим строки с пропусками
new_df = new_df.dropna()
# Исправим неточности в целевой переменной
print(new_df['Target'].value_counts())
new_df['Target'] = new_df['Target'].str.strip(" ")
new_df['Target'] = new_df['Target'].map({">50K.":">50K", 
                                       "<=50K.":"<=50K"}).fillna(new_df['Target'])
print(new_df['Target'].value_counts())

 <=50K     22654
 <=50K.    11360
 >50K       7508
 >50K.      3700
Name: Target, dtype: int64
<=50K    34014
>50K     11208
Name: Target, dtype: int64


# Задания

ВНИМАНИЕ!!!
ВЕЗДЕ, где есть параметр **random_state**, устанавливайте его равным **своему номеру**, иначе у нас могут не совпасть результаты и будет плохо (кроме KFold кросс-валидации у нас будет параметр Shufle=False, а значит и random_state не нужен). 

Результаты округляем до 5 знака после запятой. Например, ROC AUC = 0.56156

Задание *найти оптимальный гиперпараметр* (если не оговорено иное) подразумевает 5  фолдовую стратифицированную кросс-валидацию с random_state, равным **вашему номеру**.

По умолчанию мы используем ВСЕ ПРИЗНАКИ из датасетов.

In [3]:
students_random_state = {
 'Базуева Мария Дмитриевна': 993,
 'Бориско Данила Ильич': 1956,
 'Братков Герман Сергеевич': 210,
 'Орлан Суван-оол': 211,
 'Валл Федор Викторович': 188,
 'Егорова Анна Сергеевна': 25,
 'Едовина Алина Игоревна': 35,
 'Загарнюк Елизавета Максимовна': 979,
 'Захаров Алексей Сергееивч': 587,
 'Калёнов Алексей Аркадьевич': 1334,
 'Карасева Алина Александровна': 1265,
 'Каширин Егор Михайлович': 1672,
 'Косинов Андрей Вячеславович': 940,
 'Красиков Евгений Владимирович': 601,
 'Кузьмин Никита Кириллович': 452,
 'Монгуш Тенгиз Анатольевич': 668,
 'Мурадян Акоп Араратович': 1155,
 'Наумова Анастасия Юрьевна': 1020,
 'Панчук Александр Сергеевич': 1125,
 'Пашинина Татьяна Викторовна': 268,
 'Пустоваров Артем Андреевич': 187,
 'Роговая Тамара Олеговна': 1472,
 'Селезнев Дмитрий Владимирович': 734,
 'Сидорякин Виталий Дмитриевич': 1554,
 'Филиппов Антон Павлович': 126,
 'Фрольцов Григорий Максимович': 1723,
 'Хамитов Давид Альбертович': 1944,
 'Хомушку Ганна Алексеевна': 582,
 'Царева Мария Сергеевна': 1336}


In [4]:
# Введите свой номер random state
your_number_random_state = 25

# Для расчета доли, по которой будем делить датасеты
max_ = max(list(map(lambda x: np.log(x), students_random_state.values())))
frac_student = np.log(your_number_random_state) / max_

# Уникальный датасет для каждого студента
df_model = new_df.sample(frac = frac_student, random_state=your_number_random_state)


# Определим целевую переменную 
X, y = df_model.drop("Target", axis=1), df_model['Target'].map({">50K":1, "<=50K":0})

In [5]:
# Разобьем на тренировочную и тестовую (уже уменьшенный датасет для каждого студента)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                   random_state=your_number_random_state)

# Стандартизируем количественные признаки

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[quant_cols] = sc.fit_transform(X_train[quant_cols])
X_test[quant_cols] = sc.transform(X_test[quant_cols])

fio = 'Валл Федор Викторович'
final_df = pd.DataFrame(index=[fio], columns=np.arange(1, 11))
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

1) Обучите случайный лес с параметрами по умолчанию и со всеми признаками и
 сделайте прогноз на тестовых данных. Чему равен `Precision` на тестовой выборке?

2) Среди следующих значений коэффициентов регуляризации альфа выберите тот,
для которого `Recall` score на тестовой выборке для метода LightGBM c числом деревьев =
50 и остальными опциями по умолчанию *со всеми факторами* будет максимальным.
В ответе укажите значение максимального `Recall`.
Используем библиотеку lightgbm и метод LGBMClassifier.

$$ regalpha = [0.01, 0.05, 0.1, 0.3, 0.5, 0.75, 0.9, 1] $$

3) Чему равен `ROC AUC` для алгоритма XGBoost с параметрами по умолчанию
(реализация библиотеки xgboost метода XGBClassifier, random_state соответствует
вашему номеру) и с признаками, корреляция (по Пирсону) которых с целевой
переменной больше 0.15 по модулю?


4) Сделайте прогноз на тестовой выборке при помощи алгоритма Catboost (метода
CatBoostRegressor и через Pool для теста и трейна) с количеством итераций,
равным 50. В ответ запишите полученный `Recall`.

5) При помощи кросс-валидации найдите оптимальные параметры
*max_depth* (значения [1,3,7,10,15,20]) и *min_samples_leaf* (значения [1,5,10,15,20])
для случайного леса (остальные параметры по умолчанию).
В ответ запишите значение `ROC AUC` на тестовой выборке при оптимальных параметрах.
Оптимизироваться нужно именно на ROC AUC.

Вам может помочь эта ссылка:
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

6) Ко всем количественным признакам примените метод главных компонент с тремя
компонентами. В ответ запишите `долю дисперсии, которая была объяснена данными
компонентами`. Здесь тренировочную и тестовую выборку объединяем в одну.

7) Меняем задачу. В этом и следующем задании будем строить регрессии c объединенной выборкой. В качестве
целевой переменной берем *Age*. Теперь *Target* становится признаком. Используя все
переменные, постройте регрессию главных компонент (PCR). Используйте 10 главных компонент и регрессию
без регуляризации.
В ответ запишите `коэффициент детерминации` in-sample (sklearn.metrics.explained_variance_score).

8) Используя условия предыдущего пункта, постройте регрессию методом частных
наименьших квадратов (PLS) с количеством компонент = 10.
В ответ запишите `коэффициент детерминации` in-sample.
(sklearn.metrics.explained_variance_score).

9) Используя изначальные признаки (забываем про 7,8 задание), примените метод
kMeans с параметрами по умолчанию для числа кластеров
*n_cluster* = {2,3,4,5,6,7,8,9,10} на объединенной выборке (трейн + тест). В ответ запишите наибольшее значение
`квадратного корня из inertia`.

10) Примените метод главных компонент с количеством компонент = 5.
В качестве признаков используйте эти 5 главных компонент и обучите модели
бустингов: Catboost, LightGBM, XGBoost с числом итераций = 50 и остальными параметрами
по умолчанию. В ответ запишите максимальное значение `Recall` на тестовой выборке
среди этих 3 моделей.

